# Data Cleaning and Feature Engineering 
## Developed on 10,000 entries
## Create Percentage of Purchase vs Percentage of Return by SKU
## Create Average Percent Discount by SKU

In [1]:
import pandas as pd
import numpy as np
import statistics as stat
# dat_chunk = pd.read_csv('202004_to_202007.csv', chunksize=100000, iterator=True)
# dat = pd.concat(dat_chunk, ignore_index=True)
dat_chunk = pd.read_csv('trnsact_full.csv', chunksize = 500000, iterator = True, header=None)
dat = pd.concat(dat_chunk, ignore_index = True)
print(dat.shape)
dat.head(10)

(120916896, 14)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,5723053,5504,750,4000,0,2004-08-27,P,1,59.5,27.99,27.99,909200099,205,0
1,5723053,5604,450,400,908806914,2004-09-08,P,1,59.5,59.50,59.50,654500046,205,0
2,5723053,5604,450,2300,949105853,2004-08-05,R,1,59.5,59.50,59.50,244800053,205,0
3,5723053,5804,330,5900,0,2004-08-28,P,1,59.5,27.99,27.99,540900139,205,0
4,5723053,6004,690,2700,0,2004-08-13,P,1,59.5,39.99,39.99,291100054,205,0
5,5723053,6004,730,3700,0,2004-08-07,P,1,59.5,39.99,39.99,592700100,205,0
6,5723053,6104,770,1900,0,2004-09-06,P,1,59.5,14.87,14.87,659800120,205,0
7,5723053,6204,70,200,0,2004-09-06,P,1,59.5,14.75,14.75,659900120,205,0
8,5723053,6404,390,3700,0,2004-08-21,P,1,59.5,39.99,39.99,915400087,205,0
9,5723053,6704,520,1600,0,2004-08-28,P,1,59.5,27.99,27.99,540800139,205,0


In [2]:
dat.columns = ["sku", "store", "register", "trannum", "seq", "saledate", "stype", "quantity", "orgprice", "amt", "amt2", "interid", "mic", "zero"]

In [3]:
dat.head(10)

,sku,store,register,trannum,seq,saledate,stype,quantity,orgprice,amt,amt2,interid,mic,zero
0,5723053,5504,750,4000,0,2004-08-27,P,1,59.5,27.99,27.99,909200099,205,0
1,5723053,5604,450,400,908806914,2004-09-08,P,1,59.5,59.50,59.50,654500046,205,0
2,5723053,5604,450,2300,949105853,2004-08-05,R,1,59.5,59.50,59.50,244800053,205,0
3,5723053,5804,330,5900,0,2004-08-28,P,1,59.5,27.99,27.99,540900139,205,0
4,5723053,6004,690,2700,0,2004-08-13,P,1,59.5,39.99,39.99,291100054,205,0
5,5723053,6004,730,3700,0,2004-08-07,P,1,59.5,39.99,39.99,592700100,205,0
6,5723053,6104,770,1900,0,2004-09-06,P,1,59.5,14.87,14.87,659800120,205,0
7,5723053,6204,70,200,0,2004-09-06,P,1,59.5,14.75,14.75,659900120,205,0
8,5723053,6404,390,3700,0,2004-08-21,P,1,59.5,39.99,39.99,915400087,205,0
9,5723053,6704,520,1600,0,2004-08-28,P,1,59.5,27.99,27.99,540800139,205,0


In [4]:
# Replacing 0 values in orgprice and amt as the mean of the orgprice and amt of that specific sku
dat['orgprice'] = dat['orgprice'].replace(0, np.nan)
dat['orgprice'] = dat['orgprice'].fillna(dat.groupby(['sku'])['orgprice'].transform('mean'))

dat['amt'] = dat['amt'].replace(0, np.nan)
dat['amt'] = dat['amt'].fillna(dat.groupby(['sku'])['amt'].transform('mean'))
dat.head(10)

,sku,store,register,trannum,seq,saledate,stype,quantity,orgprice,amt,amt2,interid,mic,zero
0,5723053,5504,750,4000,0,2004-08-27,P,1,59.5,27.99,27.99,909200099,205,0
1,5723053,5604,450,400,908806914,2004-09-08,P,1,59.5,59.50,59.50,654500046,205,0
2,5723053,5604,450,2300,949105853,2004-08-05,R,1,59.5,59.50,59.50,244800053,205,0
3,5723053,5804,330,5900,0,2004-08-28,P,1,59.5,27.99,27.99,540900139,205,0
4,5723053,6004,690,2700,0,2004-08-13,P,1,59.5,39.99,39.99,291100054,205,0
5,5723053,6004,730,3700,0,2004-08-07,P,1,59.5,39.99,39.99,592700100,205,0
6,5723053,6104,770,1900,0,2004-09-06,P,1,59.5,14.87,14.87,659800120,205,0
7,5723053,6204,70,200,0,2004-09-06,P,1,59.5,14.75,14.75,659900120,205,0
8,5723053,6404,390,3700,0,2004-08-21,P,1,59.5,39.99,39.99,915400087,205,0
9,5723053,6704,520,1600,0,2004-08-28,P,1,59.5,27.99,27.99,540800139,205,0


In [5]:
# Note that quantity is 1, so we do not need to take into account the number of returns / purchases per transaction
percent_purchase = dat.groupby(['sku', 'stype'], group_keys=False)["stype"].aggregate(['count']).reset_index()
purchase_return = percent_purchase.pivot(index = 'sku', columns = 'stype', values = 'count').reset_index().fillna(0)
purchase_return["Percentage Purchase"] = purchase_return.apply(lambda x: 100 * x.P / (x.P + x.R), axis = 1)
purchase_return["Percentage Return"] = purchase_return.apply(lambda x: 100 * x.R / (x.P + x.R), axis = 1)
purchase_return.head(10)

stype,sku,P,R,Percentage Purchase,Percentage Return
0,3,32.0,8.0,80.000000,20.000000
1,4,26.0,3.0,89.655172,10.344828
2,5,20.0,2.0,90.909091,9.090909
3,8,6.0,1.0,85.714286,14.285714
4,20,1.0,0.0,100.000000,0.000000
5,23,5.0,1.0,83.333333,16.666667
6,24,28.0,1.0,96.551724,3.448276
7,50,0.0,1.0,0.000000,100.000000
8,55,7.0,0.0,100.000000,0.000000
9,65,3.0,0.0,100.000000,0.000000


In [6]:
purchase_return.to_csv("purchase_return.csv", index = False)

In [6]:
percent_discount = dat.groupby(['sku']).apply(lambda x: (100 * (x.orgprice - x.amt) / x.orgprice).mean()).reset_index()
discount = pd.DataFrame(percent_discount).rename(columns={0: "Percent Discount"})
discount.head(10)

MemoryError: Unable to allocate 923. MiB for an array with shape (120916896,) and data type int64

In [1]:
min(dat["sku"])

NameError: name 'dat' is not defined